In [21]:
import pandas as pd

In [22]:
df_old_metrics: pd.DataFrame = pd.read_csv('../data/embedded_datasets/metrics_dataset.csv')
print(df_old_metrics.shape)
df_old_metrics.head()

(2159, 51)


,sample_id,severity,cbo,wmc,dit,rfc,lcom,tcc,lcc,totalMethodsQty,...,innerClassesQty,lambdasQty,uniqueWordsQty,modifiers,logStatementsQty,from_project,label,type_class,type_innerclass,type_interface
0,4432196,major,3,12,1,0,54,0.0,0.0,12,...,0,0,21,1,0,1,1,1.0,0.0,0.0
1,7391055,major,3,0,1,0,0,-1.0,-1.0,0,...,0,0,12,1,0,1,1,1.0,0.0,0.0
2,4687786,major,6,28,1,16,14,0.0,0.0,17,...,0,0,96,1,0,1,1,1.0,0.0,0.0
3,3797964,major,0,0,1,0,0,-1.0,-1.0,0,...,0,0,5,9,0,1,1,0.0,1.0,0.0
4,5339993,major,2,3,2,0,0,0.0,0.0,3,...,0,0,10,1,0,1,1,1.0,0.0,0.0


In [23]:
df_pmd: pd.DataFrame = pd.read_csv('../data/embedded_datasets/mlcq_pmd.csv')
df_pmd = df_pmd.loc[df_pmd['smell'] == 'data class']
df_pmd = df_pmd.loc[df_pmd['language'] == 'java']
df_pmd = df_pmd.dropna(axis='columns')
df_pmd = df_pmd.drop(columns=['id','reviewer_id','col1','smell','review_timestamp','type','name','analysis_timestamp','name..10','language','repository','commit_hash','sample_id..14','severity'])
df_pmd = df_pmd.drop_duplicates()
print(df_pmd.shape)
df_pmd.head()

(2338, 19)


,sample_id,abc_size,accessors,attributes,average_method_complexity,average_not_accessor_or_mutator_method_complexity,children,functions_m,instability,instance_variables,lines_of_code,methods,methods_weighted,mutators,not_accessors_or_mutators,not_accessors_or_mutators_weighted,tree_impurity,used_by,uses
0,3698323,5.162278,0.0,0.0,0.250000,0.250,11.0,4.0,1.000000,0.0,13,4.0,1.0,0.0,4.0,1.0,0.0,0.0,2.0
13,3699849,0.000000,0.0,0.0,0.000000,0.000,11.0,6.0,0.166667,0.0,6,6.0,0.0,1.0,5.0,0.0,0.0,5.0,1.0
19,3700666,299.532471,25.0,10.0,1.052632,1.275,101.0,80.0,0.176471,10.0,746,76.0,80.0,11.0,40.0,51.0,0.0,28.0,6.0
32,3702984,281.724039,0.0,0.0,0.000000,0.000,52.0,35.0,0.333333,0.0,519,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
41,3705164,21.678748,3.0,3.0,0.800000,2.000,24.0,13.0,0.050000,3.0,68,10.0,8.0,3.0,4.0,8.0,0.0,19.0,1.0


In [24]:
df = pd.merge(df_old_metrics, df_pmd, on='sample_id')
print(df.shape)
df.head()

(2159, 69)


,sample_id,severity,cbo,wmc,dit,rfc,lcom,tcc,lcc,totalMethodsQty,...,instance_variables,lines_of_code,methods,methods_weighted,mutators,not_accessors_or_mutators,not_accessors_or_mutators_weighted,tree_impurity,used_by,uses
0,4432196,major,3,12,1,0,54,0.0,0.0,12,...,6.0,36,12.0,0.0,6.0,0.0,0.0,0.0,3.0,1.0
1,7391055,major,3,0,1,0,0,-1.0,-1.0,0,...,5.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4687786,major,6,28,1,16,14,0.0,0.0,17,...,7.0,109,15.0,9.0,1.0,7.0,9.0,0.0,24.0,1.0
3,3797964,major,0,0,1,0,0,-1.0,-1.0,0,...,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
4,5339993,major,2,3,2,0,0,0.0,0.0,3,...,2.0,12,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [25]:
pd.to_pickle(df, '../data/embedded_datasets/metrics_pmd_merged.pkl')
del df, df_old_metrics, df_pmd

# Merge embedding

In [26]:
df_metrics = pd.read_pickle('../data/embedded_datasets/metrics_pmd_merged.pkl')

aggregations = ['whole', 'avg', 'sum']
for agg in aggregations:
    df_t5p = pd.read_pickle(f'../data/embedded_datasets/T5_plus_{agg}.pkl')
    df_t5p = df_t5p.drop(columns=['severity', 'label'])
    df = pd.merge(df_t5p, df_metrics, on='sample_id')
    pd.to_pickle(df, f'./embedded_datasets/{agg}_pmd_merged.pkl')

In [27]:
df = pd.read_pickle('./embedded_datasets/whole_merged.pkl')
df.head()

,sample_id,em_1,em_2,em_3,em_4,em_5,em_6,em_7,em_8,em_9,...,innerClassesQty,lambdasQty,uniqueWordsQty,modifiers,logStatementsQty,from_project,label,type_class,type_innerclass,type_interface
0,4432196,-0.032667,-0.014135,-0.026271,0.013122,0.006806,0.014888,0.037827,-0.149637,0.005567,...,0,0,21,1,0,1,1,1.0,0.0,0.0
1,7391055,-0.016414,0.004841,-0.074231,-0.010195,-0.077366,-0.055006,0.051985,0.008177,0.018444,...,0,0,12,1,0,1,1,1.0,0.0,0.0
2,4687786,-0.018041,-0.004828,-0.164243,-0.003471,-0.006314,0.011207,-0.058506,-0.069209,0.018128,...,0,0,96,1,0,1,1,1.0,0.0,0.0
3,3797964,-0.097944,0.048423,-0.008628,-0.022152,-0.070442,-0.049730,-0.064467,-0.026823,0.038378,...,0,0,5,9,0,1,1,0.0,1.0,0.0
4,5339993,-0.006522,0.013563,-0.120788,-0.006446,-0.051390,-0.048318,0.045672,-0.069744,0.078559,...,0,0,10,1,0,1,1,1.0,0.0,0.0
